In [ ]:
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
def KNNPCA(Kv,Ka,Components,UserName,Window):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+Window+UserName+'train.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
       	
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
        
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
        	
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Ka)
    trained_model=clf.fit(X_train,z_train)
    trained_model.fit(X_train,z_train )
    
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
#     print(z_predict)
#     print(collections.Counter(ArousalFile))
#     print(collections.Counter(ValenceFile))
#     if accuracy_score(z_test, z_predict)*100==75:
# #         print(np.count_nonzero(z_predict))
#         print(z_predict)
#         MainXTrain=X_train
#         MainZTrain=z_train
#         BandsFile = np.genfromtxt('training/s01train.csv',delimiter=',')
#         pca = PCA(n_components=Components, svd_solver='full')
#         pca.fit(BandsFile)
#         BandsFile = pca.transform(BandsFile)
#         Accurecies=[]
#         for i in range(10):
#             clf.fit(MainXTrain,MainZTrain)
#             file_path = os.path.join("ChannelsExtracted","s03", 'ArousalEncoded'+".csv")
#             ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',')
#             file_path = os.path.join("ChannelsExtracted","s03", 'ValenceEncoded'+".csv")
#             ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',')
#             BandsFile = np.genfromtxt('training/s03train.csv',delimiter=',')
#             X_train0, X_test0, y_train, y_test, z_train0, z_test0 = train_test_split(BandsFile, ValenceFile,ArousalFile ,test_size=0.2)
#             z_predict_repeat = clf.predict(X_test0)
# #             print(z_test0)
#             Accurecies+=[accuracy_score(z_test0,z_predict_repeat)]
#         print("100% Test "+str(np.mean(Accurecies)))
#         print(collections.Counter(Accurecies))
#     if accuracy_score(z_test, z_predict)*100==100:
#         print( z_predict)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100]
#     cm = confusion_matrix(y_test, y_predict)
#     print(cm)
    
#     print("Accuracy score of Arousal test KNN-PCA")
#     print(accuracy_score(y_test, y_predict)*100)
    
def Run1000(Kv,Ka,Components,User,Window):
    arousal=[]
    valence=[]
    for i in range (100):
        knearest=KNNPCA(Kv,Ka,Components,User,Window)
        arousal+=[knearest[0]]
        valence+=[knearest[1]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
if __name__ == "__main__":
    for i in range(9):
        print("KnearestPCA S0"+str(i+1))
        Run1000(3,5,20,"S0"+str(i+1),"256")
    for i in range(10,28):
        print("KnearestPCA S"+str(i))
        Run1000(3,5,20,"S"+str(i),"256")
#     for i in range(10):
#         Run1000(3,5,20,"S03","256")

KnearestPCA S01
Arousal 54.875
Counter({50.0: 28, 62.5: 28, 37.5: 22, 75.0: 13, 87.5: 4, 25.0: 3, 12.5: 1, 100.0: 1})
Valence 47.5
Counter({50.0: 29, 37.5: 27, 62.5: 20, 25.0: 11, 75.0: 6, 12.5: 4, 87.5: 3})
KnearestPCA S02
Arousal 63.75
Counter({62.5: 35, 75.0: 30, 50.0: 15, 37.5: 10, 87.5: 9, 25.0: 1})
Valence 60.75
Counter({62.5: 30, 75.0: 28, 50.0: 20, 37.5: 14, 87.5: 6, 25.0: 2})
KnearestPCA S03
Arousal 73.75
Counter({75.0: 36, 62.5: 26, 87.5: 26, 50.0: 6, 100.0: 4, 37.5: 2})
Valence 74.0
Counter({75.0: 33, 87.5: 26, 62.5: 16, 50.0: 13, 100.0: 9, 37.5: 2, 25.0: 1})
KnearestPCA S04
Arousal 44.75
Counter({50.0: 27, 37.5: 26, 25.0: 19, 62.5: 17, 12.5: 4, 75.0: 4, 87.5: 3})
Valence 51.5
Counter({62.5: 31, 50.0: 31, 37.5: 21, 75.0: 10, 25.0: 4, 12.5: 2, 0.0: 1})
KnearestPCA S05
Arousal 61.0
Counter({62.5: 31, 50.0: 24, 75.0: 24, 37.5: 10, 87.5: 7, 25.0: 3, 100.0: 1})
Valence 56.5
Counter({62.5: 25, 50.0: 25, 75.0: 22, 37.5: 14, 25.0: 9, 87.5: 5})
KnearestPCA S06
Arousal 41.0
Counter({3